In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import scipy
from pathlib import Path
from scipy.signal import lfilter, firwin

sys.path.append('../../src/')
from utils import analysis_utils as analysis
from utils import plotting_utils as plotting
import datetime
## Data settings
from statannot import add_stat_annotation

## Plotting settings
import matplotlib.patches as mpatches
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

plt.rcParams['font.sans-serif'] = "Arial"
plt.rcParams['font.family'] = "sans-serif"

sns.set_context("talk")

import ipywidgets as widgets
from IPython.display import display
from matplotlib.patches import Rectangle

# Explore the sniff sensor data with the lick sensor shorting. 

Associate document exploring these details: https://alleninstitute-my.sharepoint.com/:w:/g/personal/tiffany_ona_alleninstitute_org/EQmEAcuGsHxAqWb2WePXJxoB2nmKrqrQ0Cp-3Xy0Yq0NlQ?e=QFFLbp


In [ ]:
date = datetime.date.today()
date_string = "4/12/2024"
dacte = datetime.datetime.strptime(date_string, "%m/%d/%Y").date()

In [ ]:
pdf_path = r'Z:\scratch\vr-foraging\sessions'
base_path = 'Z:/scratch/vr-foraging/data/'

In [ ]:
mouse = '716458'

In [ ]:
session_found = False

directory = os.path.join(base_path, mouse)
files = os.listdir(os.path.join(base_path, mouse))

sorted_files = sorted(files, key=lambda x: os.path.getctime(os.path.join(directory, x)), reverse=True)

for file_name in sorted_files:
    
    print(file_name)
    session_path = os.path.join(base_path, mouse, file_name)
    session = file_name[:8]
    session_path = Path(session_path)
    
    if session_found == True:
        break
    
    if datetime.date.fromtimestamp(os.path.getctime(session_path)) != date:
        continue
    else:
        print('correct date found')
        session_found = True
    

    data = analysis.load_session_data(session_path)
    

    data['harp_olfactometer'].streams.OdorValveState.load_from_file()
    data['harp_olfactometer'].streams.EndValveState.load_from_file()
    data['software_events'].streams.ActiveSite.load_from_file()
    data['software_events'].streams.ChoiceFeedback.load_from_file()
    
    data['harp_behavior'].streams.OutputSet.load_from_file()
    data['harp_behavior'].streams.OutputClear.load_from_file()
    
    reward_sites, active_site, encoder_data, config = analysis.parse_data(data)
    
    data['harp_lickometer'].streams.LickState.load_from_file()
    licks = data['harp_lickometer'].streams.LickState.data['Channel0'] == True
    licks = licks.loc[licks == True]

### Try the signal from the sniff sensor

In [ ]:
def plot_thermistor(x_start):
    fig, ax = plt.subplots(2,1, figsize=(10,4), sharex=True, gridspec_kw={'height_ratios': [3, 1]})
    plt.suptitle(f'{mouse}')
    if 'harp_sniffsensor' in data:
        print('plot harp sniff sensor')
        data['harp_sniffsensor'].streams.RawVoltage.load_from_file()
        zero_index = data['harp_sniffsensor'].streams.RawVoltage.data['RawVoltage'].index[0]
        ax[0].plot(data['harp_sniffsensor'].streams.RawVoltage.data['RawVoltage'].index-zero_index, data['harp_sniffsensor'].streams.RawVoltage.data['RawVoltage'].values)
        sns.despine()
        
    else:
        print('plot analog sniff sensor')

        zero_index = data['harp_behavior'].streams.AnalogData.data['AnalogInput0'].index[0]
        ax[0].plot(data['harp_behavior'].streams.AnalogData.data['AnalogInput0'].index-zero_index, data['harp_behavior'].streams.AnalogData.data['AnalogInput0'].values)

    ax[0].set_xlim([x_start, x_start + 5])
    ax[0].set_ylabel('Voltage (V)')
    
    ax[1].scatter(licks.index - zero_index,  licks.index * 0 + 1, marker='|')
    ax[1].set_xlim([x_start, x_start + 5])
    ax[1].set_xlabel('Time (s)')
    sns.despine()
    plt.tight_layout()

        
# Define callback functions for the arrow buttons
def on_left_button_clicked(button):
    x_start_widget.value -= 5

def on_right_button_clicked(button):
    x_start_widget.value += 5

# Create arrow buttons
left_button = widgets.Button(description='◄')
right_button = widgets.Button(description='►')

# Define widget for the starting value of x-axis
x_start_widget = widgets.FloatText(value=00.0, description='X start:', continuous_update=False)

# Set button click event handlers
left_button.on_click(on_left_button_clicked)
right_button.on_click(on_right_button_clicked)

# Arrange the buttons and widget horizontally
button_box = widgets.HBox([left_button, right_button])
ui = widgets.VBox([button_box, x_start_widget])

# Create interactive plot
interactive_plot = widgets.interactive_output(plot_thermistor, {'x_start': x_start_widget})

# Display the interactive plot and UI
display(ui, interactive_plot)


### Plot both signals acquired at the same time

In [ ]:
date = datetime.date.today()
date_string = "4/17/2024"
dacte = datetime.datetime.strptime(date_string, "%m/%d/%Y").date()

In [ ]:
pdf_path = r'Z:\scratch\vr-foraging\sessions'
base_path = 'Z:/scratch/vr-foraging/data/'

In [ ]:
mouse = '716455'

In [ ]:
def plot_thermistor(x_start):
    fig, ax = plt.subplots(3,1, figsize=(10,7), sharex=True, gridspec_kw={'height_ratios': [3,3, 1]})
    plt.suptitle(f'{mouse}')

    print('plot harp sniff sensor')
    data['harp_sniffsensor'].streams.RawVoltage.load_from_file()
    zero_index = data['harp_sniffsensor'].streams.RawVoltage.data['RawVoltage'].index[0]
    ax[0].plot(data['harp_sniffsensor'].streams.RawVoltage.data['RawVoltage'].index-zero_index, data['harp_sniffsensor'].streams.RawVoltage.data['RawVoltage'].values, label = 'Harp Sniff Sensor', linewidth=1.5)
    ax[0].set_ylim(2050,2150)
    print('plot analog sniff sensor')

    zero_index = data['harp_behavior'].streams.AnalogData.data['AnalogInput0'].index[0]
    ax[1].plot(data['harp_behavior'].streams.AnalogData.data['AnalogInput0'].index-zero_index, data['harp_behavior'].streams.AnalogData.data['AnalogInput0'].values, color='indigo', label = 'Analog Sniff Sensor', linewidth=1)
    ax[1].set_ylim(1250,1350)
    ax[0].set_xlim([x_start, x_start + 5])

    ax[0].set_ylabel('Voltage (V)')
    
    ax[2].scatter(licks.index - zero_index,  licks.index * 0 + 1, marker='|')
    ax[2].set_xlim([x_start, x_start + 5])
    ax[2].set_xlabel('Time (s)')
    sns.despine()
    plt.tight_layout()
    plt.legend(handles=[mpatches.Patch(color='blue', label='Harp Sniff Sensor'), mpatches.Patch(color='indigo', label='Analog Sniff Sensor')], bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

# Define callback functions for the arrow buttons
def on_left_button_clicked(button):
    x_start_widget.value -= 5

def on_right_button_clicked(button):
    x_start_widget.value += 5

# Create arrow buttons
left_button = widgets.Button(description='◄')
right_button = widgets.Button(description='►')

# Define widget for the starting value of x-axis
x_start_widget = widgets.FloatText(value=00.0, description='X start:', continuous_update=False)

# Set button click event handlers
left_button.on_click(on_left_button_clicked)
right_button.on_click(on_right_button_clicked)

# Arrange the buttons and widget horizontally
button_box = widgets.HBox([left_button, right_button])
ui = widgets.VBox([button_box, x_start_widget])

# Create interactive plot

interactive_plot = widgets.interactive_output(plot_thermistor, {'x_start': x_start_widget})

# Display the interactive plot and UI
display(ui, interactive_plot)

